# OMOP to Patient Follow Up IDEA4RC

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

In [ ]:
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

# PROBLEMS

Here we do not have the code for the new cancer diagnosis in omop, so we can not find the date of the last cancer diagnosis. We need to check this. If this is in observation, then we need the Variable as Modifier.

For the ALIVE, where do we find this data in OMOP? DEAD is in DEATH, even if we do not have the data of the reason (other caouse and unknown cause in OMOP???)

How to "new diagnosis"??? 




In [ ]:
#I will simply use the codes episode_concept_id=32533 and episode_type_concept_id=32879

query= """
    SELECT p.person_id
    FROM person p;
"""
curOMOP.execute(query)
insert= """
    INSERT INTO patient_follow_up (patient, date_of_new_cancer_diagnosis, last_contact, new_cancer_diagnosis, new_cancer_topography, patient_follow_up_date,status_at_last_follow_up)
    VALUES (%s, %s, %s, %s, %s, %s);
"""

for person in persons:
    person_id = person[0]
    queryDeath = """
        SELECT death_date
        FROM death
        WHERE person_id = %s;
    """
    curOMOP.execute(queryDeath, (person_id))
    death_date = curOMOP.fetchone()
    status_last_FU=""
    if death_date:
        status_last_FU="DEATH"      #FOR DEATH, HOW TO GET THE CAUSE OF DEATH?
    else:    #this case is more complicated
        queryAlive= """
            SELECT episode_start_date,  episode_concept_id
            FROM episode
            WHERE person_id = %s AND episode_concept_id IN (2946,32947,32949,32948)
            AND episode_start_date>=(SELECT MAX (episode_start_date) FROM episode WHERE person_id = %s AND episode_concept_id IN (2946,32947,32949,32948) LIMIT 1);
            LIMIT 1;´
        """
        curOMOP.execute(queryAlive, (person_id,person_id))
        last_episode = curOMOP.fetchone()
        if last_episode:
            status_last_FU = "NED" if last_episode[1] == 32946 else "AWD"
    
    dateQuery= """
        SELECT MAX (episode_start_date), MAX (episode_end_date)
        FROM episode
        WHERE person_id = %s;
    """
    curIDEA.execute(dateQuery, (person_id))
    last_episode = curIDEA.fetchone()
    patient_follow_up_date = np.max(last_episode[0], last_episode[1])

    dateVisit= """
        SELECT MAX(visit_start_date)
        FROM visit_occurrence
        WHERE person_id = %s;
    """
    curOMOP.execute(dateVisit, (person_id))
    last_visit = curOMOP.fetchone()
    last_visit_date = last_visit[0]
    last_FU=np.max(last_visit_date, patient_follow_up_date)
    curIDEA.execute(insert, (person_id, ???, lastFU, ???,???,  last_visit_date, status_last_FU))

    
    

        
        
